#init script to install and configure the Splunk UF on cluster members

In [2]:
dbutils.fs.put("dbfs:/splunkUF/splunkUF-init.sh" ,"""
#https://github.com/rzzldzzl/Install-Splunk-UF-on-Databricks-Cluster
ufDlUrl="https://download.splunk.com/products/universalforwarder/releases/8.0.2.1/linux/splunkforwarder-8.0.2.1-f002026bad55-Linux-x86_64.tgz"
ufDlDir="/dbfs/splunkUF"
ufInstallDir="/local_disk0"
TARGETURI="<FQDN of Splunk Deployment Server>:8089"

set -e
ufDlFilename=$(basename "${ufDlUrl}")
ufDlPath="${ufDlDir}/${ufDlFilename}"
LCKFILE="${ufDlDir}/.LCKFILE"
SPLUNK_HOME="${ufInstallDir}/splunkforwarder"

# sleep if download lock file exists
sleep 0.$[ ( $RANDOM % 1000 ) ]s
i=0
while [ -e ${LCKFILE} ] && [ $i -lt 30 ]; do
  echo "#### Lock file exists.  Sleeping while ${ufDlFilename} is downloaded to ${ufDlDir} by $(cat ${LCKFILE})####"
  let i=i+1
  sleep 1
done

# If the splunk unervisal forwarder installer does not exist, download it from the splunk website
if ! [ -e ${ufDlPath} ]; then
  mkdir -p ${ufDlDir}
  printf $(hostname) > ${LCKFILE}
  echo "#### Downloading ${ufDlFilename} to ${ufDlDir} ####"
  set +e
  curl -s "${ufDlUrl}" -o ${ufDlPath} || { rm ${LCKFILE}; exit 1; }
  curl -s "${ufDlUrl}.md5" -o ${ufDlPath}.md5 || { rm ${LCKFILE}; exit 1; }
  rm ${LCKFILE}
  set -e
fi

#Install Splunk UF
echo "#### Installing from ${ufDlPath} ####"
(cd ${ufDlDir} && md5sum -c "${ufDlFilename}.md5")
mkdir -p "${ufInstallDir}"
[ -e "${SPLUNK_HOME}" ] && rm -rf "{SPLUNK_HOME}"
tar -xzf ${ufDlPath} -C "${ufInstallDir}"

#Configure deployment client
mkdir -p ${SPLUNK_HOME}/etc/apps/local/default
cat > ${SPLUNK_HOME}/etc/apps/local/default/deploymentclient.conf << EOF
[deployment-client]
clientName = dbr-${HOSTNAME}
[target-broker:deploymentServer]
targetUri = ${TARGETURI}
EOF

#Add DB_CLUSTER_ID & DB_IS_DRIVER as indexed fields/meta data to the events
cat > ${SPLUNK_HOME}/etc/apps/local/default/inputs.conf << EOF
[default]
_meta = DB_CLUSTER_ID::${DB_CLUSTER_ID} DB_IS_DRIVER::${DB_IS_DRIVER}
EOF

${SPLUNK_HOME}/bin/splunk start --accept-license --answer-yes --no-prompt
""", True)